# **1.a**

#### **State space**

In a finite MPD, a **state space** is defined as all of the possible configurations the agent can be in given its environment. To create a state space we need to define all the pieces of important information to represent the MPD problem. I took into account the elevator, elevator floor, door status, call floors, exit floors, and location of the person. So we can say:

$State\_Space = $

$((E_{A}, F_{A_i}, D_{A_k}), (E_{B}, F_{B_j}, D_{B_l}), ((F_{Call_m}, F_{Exit_o}, F_{Location_p}), (F_{Call_n}, F_{Exit_p}, F_{Location_r})))$ 

for all $i,j,k,l,m,n,o,p,q,r$

Where:
- $E$ is one of the elevators
- $F$ is the floor of one of the elevators
- $Door$ is the status of the door (OPEN or CLOSED)
- $F_{Call}$ is the floor a person is calling from (0 if no passenger)
- $F_{Exit}$ is the floor a person wants to exit (0 if no passenger)
- $F_{Location}$ is where the person is located (IN_A, IN_B, or WAITING)
- $i,j \space \epsilon \space [1,2,3,4,5,6]$
- $m,n,o,p \space \epsilon \space [0,1,2,3,4,5,6]$ 
- $k,l \space \epsilon \space [open, closed]$
- $q,r \space \epsilon \space [in\_a, in\_b, waiting]$

The state space would be of size:

$|State\_Space| = |((1, 6, 2), (1, 6, 2), ((7, 7, 3), (7, 7, 3)))| = $

$= |1 * 6 *  2 * 1 * 6 * 2 * 7 * 7 * 3 * 7 * 7 * 3| = $

$= 3111696$ states

#### **Action space**

Simarly we define the **action space** as the unique actions that can be taken at a time step. 
Given there are 2 elevators, each elevator moves independently, and we have the action set for elevator as 
${UP, DOWN, HOLD, DOORS}$ we can define:

$Action\_Set = ['UP', 'DOWN', 'HOLD', 'DOORS']$ 

$Action\_Space = ((Action\_Set_{i} , E_{A}), (Action\_Set_{j} , E_{B}))$

for all $i,j$

Where:
- $E_{A}$ is elevator A
- $E_{B}$ is elevator B

The size of the action space is $(4 * 1 * 4 * 1) = 16 $

#### **Time step**

Lastly, we need to define a timestep. Since the elevator can only make decisions every 5 seconds, we can decide our timestep to be 5 seconds. And this timestep still holds even given the arrival rates, as we can model the arrivals rate by the number of seconds in the timestep.

# **1.b**


#### **Reward Function**

For the reward function we care about the wait times. We need to devise a way to reward the agent when it minimizes the wait time of an action and penalize it when it lengthens the wait time of a passenger. We can define the reward psuedocode function as follows:

```
def reward(s, a, s')

    reward = 0
    m_r = movment reward
    m_p = movement penalty

    d_r = door reward
    d_p = door penalty

    h_r = hold reward
    h_p = hold penalty

    given action a, current state, and new state:

        if closer to exit floor:
            reward += m_r
        else:
            penalize -= m_p

        if closer to call floor
            reward += m_r
        else:
            penalize -= m_p

        if closer doors opened and departed passenger:
            reward += d_r
        else:
            penalize -= d_p

        if hold doors and departed passengers
            reward += h_r
        else:
            penalize -= h_p

    return reward

```
This function would go through the action pairs for elevator A and B and aggregate the rewards for the corresponding action.

#### **Utility**

Additionally, generally, we typically define **Utility** as the expected sum of future rewards:

$Utility = reward(s, a, s')_{t} + \gamma_{k} * reward(s, a, s')_{t+1} +  ... + \gamma^k * reward(s, a, s')_{t + n}$

Where gamma is the discount factor. 

In terms of actual implementation of the **Utility function** in our MPD problem, we implicity define the utility of the state-actions inside a **Q table**, which is the expected future rewards of each state action pairs:

$Q[state][action] = Utility(s,a)$

Where $Utility(s,a)$ is a method to determine the expected future/expected rewards. The implementation of $Utility(s,a)$ would cahnge given on which RL algorithm being implemented.

# **1.c**

To define a simulator for the elevator system, we need to define:
- a state transition rules for each step
- a simulation for passengers
- a reward function

You can view the environment model / simulator implementation [HERE](./ENVIRONMENT/environment.py)

### **Solution 1.d**

If we wanted to model impatience, we would have to incorporate **elapsed time** to our **state space** and **reward function** (the action space, given my implementation should be unchanged).

- For our state space, we would liekly have to keep track of the time elapsed. Thw downside to this approach is that time is continous, and would likely exponentially increase the state space. Even if you discretized time, the state space would still be large.

- For our reward function, we would have to use the elapsed time we keep track of in the state space and use that to add to the aggregate rewards for a single step. we would likely change the time to a negative number and add that to the rewards to incentivize shorter times: $reward= -T_{time} + reward$

- For our action space, it should remain unchanged.

### **Problem 1 MPD**

Putting all our definition together we define our elevator MPD as :

$MPD(S, A, T, R, S_{0}) = $
- $s_a \space \epsilon \space S \space where \space s_a = ((E_{A}, F_{A_i}, D_{A_k}), (E_{B}, F_{B_j}, D_{B_l}), ((F_{Call_m}, F_{Exit_o}, F_{Location_p}), (F_{Call_n}, F_{Exit_p}, F_{Location_r})))$
- $a_a \space \epsilon \space A $ where $ a_a \space = ((Action\_Set_{i} , E_{A}), (Action\_Set_{j} , E_{B}))$
- $T(s, a, s') = P(f_{e} | f_{c})$
- $R = R(s, a, a')$
- $S_{0} = s_{a_0} = ((A, 1, 1), (B, 1, 1), ((0, 0, waiting), (0, 0, waiting)))$ 